# Language models
- 言語モデルの使用方法を提供するモジュール
- 様々な言語モデルを共通のインターフェースで使用できるようにしてくれる

## LLMs
- 1つの入力テキストに対して1つのテキスト出力を返す
- すでに Legacy なので詳細は割愛

## Chat models
- チャット形式のやり取りを入力して応答を得られる

#### 本の時点からのバージョンアップによる更新点
- 元々 langchain本体に入っていたものが、OpenAI専用パッケージ `langchain_openai` に外出しされた
    ```python
    # Before
    from langchain.chat_models import ChatOpenAI

    # After
    from langchain_openai.chat_models import ChatOpenAI
    # これでもOK
    from langchain_openai import ChatOpenAI
    ```

In [11]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

messages = [
    SystemMessage(content='あなたは素人にも簡単に作れるレシピをたくさん知っているプロの料理家です。'),
    HumanMessage(content='ユーザーに使いたい食材を質問してください。ユーザーが答えた食材を使って簡単で美味しく作れる晩御飯のレシピを教えてください。出力はマークダウン形式でお願いします。'),
    AIMessage(content='どんな食材を使いたいですか？'),
    HumanMessage(content='キャベツと玉ねぎです'),
]

result = chat(messages)
print(result.content)

素晴らしい選択です！キャベツと玉ねぎを使った簡単で美味しい晩ご飯のレシピをお伝えします。

## キャベツと玉ねぎの炒め物

### 材料:
- キャベツ: 1/4個（適量）
- 玉ねぎ: 1個
- 醤油: 大さじ2
- みりん: 大さじ1
- ごま油: 小さじ1
- ごま: 適量

### 作り方:
1. キャベツはざく切り、玉ねぎは薄切りにする。
2. フライパンにごま油を熱し、玉ねぎを炒める。
3. 玉ねぎがしんなりしたらキャベツを加えてさっと炒める。
4. 醤油とみりんを加えて全体に絡める。
5. ごまをふりかけて完成！

ぜひお試しください！


## Callbackを使ったストリーミング
- 応答結果は通常、すべての出力が出揃うまで待ってから一括で取得される
- それをストリーミングで少しずつ取得することができる

In [12]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler  # ★ポイント！！
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
    streaming=True,  # ★ポイント！！
    callbacks=[StreamingStdOutCallbackHandler()],  # ★ポイント！！
)

messages = [
    SystemMessage(content='あなたは素人にも簡単に作れるレシピをたくさん知っているプロの料理家です。'),
    HumanMessage(content='ユーザーに使いたい食材を質問してください。ユーザーが答えた食材を使って簡単で美味しく作れる晩御飯のレシピを教えてください。出力はマークダウン形式でお願いします。'),
    AIMessage(content='どんな食材を使いたいですか？'),
    HumanMessage(content='キャベツと玉ねぎです'),
]

# ここで標準出力に少しずつ結果が表示される
result = chat(messages)

素早く作れて美味しい「キャベツと玉ねぎの炒め物」のレシピをご紹介します。

### キャベツと玉ねぎの炒め物

#### 材料:
- キャベツ: 1/4個（適量）
- 玉ねぎ: 1個
- 醤油: 大さじ2
- みりん: 大さじ1
- ごま油: 小さじ1
- お好みでごまや唐辛子を加えるとアクセントになります。

#### 作り方:
1. キャベツはざく切り、玉ねぎは薄切りにする。
2. フライパンにごま油を熱し、玉ねぎを炒める。
3. 玉ねぎがしんなりしたらキャベツを加えてさっと炒める。
4. 醤油とみりんを加えて全体に絡める。
5. お皿に盛り付けて、ごまや唐辛子をふりかける。

簡単でヘルシーな一品です。お好みでご飯のお供や麺類と一緒に食べるのもおすすめです。

## async streamで頑張ってみる
- 標準出力だとアプリケーションで使いづらいため

In [13]:
import asyncio

chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

messages = [
    SystemMessage(content='あなたは素人にも簡単に作れるレシピをたくさん知っているプロの料理家です。'),
    HumanMessage(content='ユーザーに使いたい食材を質問してください。ユーザーが答えた食材を使って簡単で美味しく作れる晩御飯のレシピを教えてください。出力はマークダウン形式でお願いします。'),
    AIMessage(content='どんな食材を使いたいですか？'),
    HumanMessage(content='キャベツと玉ねぎです'),
]

# async_generator生成
res = chat.astream(messages)  # ★ポイント！！


async def stream():
    """非同期generatorから少しずつ結果を取得する"""
    result = []
    async for msg in res:  # ★ポイント！！
        text = msg.content 
        print(text, end='')
        result.append(text)
    return ''.join(result)


text = await stream()
print('\n\n\n' + '-'*70)
print(text)

キャベツと玉ねぎを使った簡単で美味しい晩ご飯のレシピをご紹介します。

## キャベツと玉ねぎの炒め物

### 材料:
- キャベツ: 1/4個
- 玉ねぎ: 1個
- 醤油: 大さじ2
- みりん: 大さじ1
- ごま油: 小さじ1
- 塩、こしょう: 適量

### 作り方:
1. キャベツはざく切り、玉ねぎは薄切りにする。
2. フライパンにごま油を熱し、玉ねぎを炒める。
3. 玉ねぎがしんなりしたらキャベツを加えてさっと炒める。
4. 醤油、みりんを加えて全体に味をなじませる。
5. 塩、こしょうで味を調えて完成。

ぜひお試しください！


----------------------------------------------------------------------
キャベツと玉ねぎを使った簡単で美味しい晩ご飯のレシピをご紹介します。

## キャベツと玉ねぎの炒め物

### 材料:
- キャベツ: 1/4個
- 玉ねぎ: 1個
- 醤油: 大さじ2
- みりん: 大さじ1
- ごま油: 小さじ1
- 塩、こしょう: 適量

### 作り方:
1. キャベツはざく切り、玉ねぎは薄切りにする。
2. フライパンにごま油を熱し、玉ねぎを炒める。
3. 玉ねぎがしんなりしたらキャベツを加えてさっと炒める。
4. 醤油、みりんを加えて全体に味をなじませる。
5. 塩、こしょうで味を調えて完成。

ぜひお試しください！
